Importing Libraries and installing the "Soup"

In [3]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import csv
import requests
import pandas as pd
import numpy as np

Reading the data from the Website given 


In [7]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table')
#Ignore cells with Boroughs defined as 'Not assigned'
lst_table = []
for line in table.find_all('tr'):
    if (line.contents[3].text != 'Not assigned'):
        lst_table.append([line.contents[1].text, line.contents[3].text, line.contents[5].text[:-1]])
df_postal = pd.DataFrame(lst_table[1:], columns=lst_table[0])
df_code = df_postal['Postcode'].unique()
df_postal.set_index('Postcode', drop=False, inplace=True)

separator=', '
for postcode in df_code:
    df_tmp = df_postal.loc[postcode]
    if(df_tmp.size > 3):
        neighbourhood = separator.join(df_tmp['Neighbourhood'])
        df_postal.loc[postcode, 'Neighbourhood'] = neighbourhood

df_postal.drop(df_postal[df_postal.Neighbourhood == 'Not assigned'].index, inplace=True)
df_postal.drop_duplicates('Postcode', inplace=True)
df_postal.reset_index(drop = True, inplace=True)
df_postal.head()        

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M9A,Etobicoke,Islington Avenue


In [8]:
df_postal.shape

(102, 3)

In [19]:
df2=pd.read_csv("Geospatial_Coordinates.csv")
df2.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_postal = pd.merge(df_postal, df2[['Postcode', 'Latitude', 'Longitude']], on='Postcode')
df_postal.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
